In [1]:
# Crea un netcdf con el viento a 925 hPa alineado con respecto
# a la fecha del onset de la temporada de lluvias.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

In [2]:
# Datos.
# viento 925 hPa.

path_d = "../data/ERA5/"
path_r = "../results/onset/"

# Cantidad de días a promediar.
d = 20

# Variable, nivel de presión y región.
vars = [ "u", "v", "sst", "olr" ]
levels = [ "925", "200" ]
region = [ "mexico" ]

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[0] + "_daily.grib"
U = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Viento meridional.
fname = region[0] + "_" + vars[1] + "_" + levels[0] + "_daily.grib"
V = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Unimos en un solo netcdf.
ds = xr.merge( ( U, V ) )

# Media móvil de precipitación centrada a 15 días.
ds = ( ds.rolling(time = d, center = True).mean() )

ds

<xarray.Dataset>
Dimensions:        (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time           (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
    valid_time     (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
Data variables:
    u              (time, latitude, longitude) float64 0.1141 0.2088 ... -7.254
    v              (time, latitude, longitude) float64 0.2844 0.2771 ... -6.321
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-03-30T17:03:22 GRIB to CDM+CF via cfgrib-0....

In [3]:
# Composite de la distribución espacial.
# viento 925 hPa.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_U_" + levels[0] + "_mean_" 
    + str(d)+ "_dias.nc" )

composite

<xarray.Dataset>
Dimensions:        (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 925
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
  * dayofyear      (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 56 57 58 59 60
    spatial_ref    int64 0
Data variables:
    u              (dayofyear, latitude, longitude) float64 1.263 ... -7.602
    v              (dayofyear, latitude, longitude) float64 0.5704 ... 0.03453

In [4]:
# Datos.
# viento 200 hPa.

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[1] + "_daily.grib"
U = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Viento meridional.
fname = region[0] + "_" + vars[1] + "_" + levels[1] + "_daily.grib"
V = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Unimos en un solo netcdf.
ds = xr.merge( ( U, V ) )

# Media móvil de precipitación centrada a 15 días.
ds = ( ds.rolling(time = d, center = True).mean() )

ds

<xarray.Dataset>
Dimensions:        (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time           (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 200
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
    valid_time     (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T0...
Data variables:
    u              (time, latitude, longitude) float64 14.27 14.44 ... 4.966
    v              (time, latitude, longitude) float64 3.405 3.078 ... 15.01
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-03-30T17:09:45 GRIB to CDM+CF via cfgrib-0....

In [5]:
# Composite de la distribución espacial.
# viento 200 hPa.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_U_" + levels[1] + "_mean_" 
    + str(d)+ "_dias.nc" )


composite

<xarray.Dataset>
Dimensions:        (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 200
  * latitude       (latitude) float64 40.0 39.75 39.5 39.25 ... 0.5 0.25 0.0
  * longitude      (longitude) float64 -120.0 -119.8 -119.5 ... -50.25 -50.0
  * dayofyear      (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 56 57 58 59 60
    spatial_ref    int64 0
Data variables:
    u              (dayofyear, latitude, longitude) float64 24.8 24.73 ... 2.129
    v              (dayofyear, latitude, longitude) float64 -4.792 ... 0.08281

In [6]:
# Temperatura superficial del océano.
fname = region[0] + "_" + vars[2] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Media móvil de precipitación centrada a 15 días.
ds = ( ds.rolling(time = d, center = True).mean() )

ds

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_sst_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:     (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time        (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:0...
    step        timedelta64[ns] 00:00:00
    surface     int64 0
  * latitude    (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude   (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
    valid_time  (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:0...
Data variables:
    sst         (time, latitude, longitude) float64 nan nan nan ... nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-03-30T17:16:22 GRIB to CDM+CF via cfgrib-0....

In [7]:
# Temperatura superficial del océano.
# Composite de la distribución espacial, variable.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_sst" + "_mean_" 
    + str(d)+ "_dias.nc" )


composite

<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step         timedelta64[ns] 00:00:00
    surface      int64 0
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    sst          (dayofyear, latitude, longitude) float64 nan nan ... nan nan

In [8]:
# OLR
fname = region[0] + "_" + vars[3] + "_daily.grib"
ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )

# Media móvil de precipitación centrada a 15 días.
ds = ( ds.rolling(time = d, center = True).mean() )

ds

Ignoring index file '/Users/rodrigo/Documents/Posgrado/Maestria/Tesis/sequia/data/ERA5/mexico_olr_daily.grib.90c91.idx' older than GRIB file


<xarray.Dataset>
Dimensions:     (time: 14965, latitude: 161, longitude: 281)
Coordinates:
  * time        (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:0...
    step        timedelta64[ns] 00:00:00
    surface     int64 0
  * latitude    (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude   (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
    valid_time  (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:0...
Data variables:
    mtnlwrf     (time, latitude, longitude) float64 -239.0 -242.2 ... -199.7
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-03-30T17:18:57 GRIB to CDM+CF via cfgrib-0....

In [9]:
# OLR.
# Composite de la distribución espacial, variable.

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Creamos un Dataset para cada año.
ds_year = []
for i in range( len(years) ):
    ds_year.append( ds.sel( time = ds.time.dt.year.isin( years[i] ) ) )
    # Convertimos alas fechas a dayofyear.
    ds_year[i] = ds_year[i].groupby( "time.dayofyear" ).mean( "time" )
    # Coordenada para concatenar.
    ds_year[i].assign_coords( {"concat": i} )
    # Corrregimos el dayofyear para los años bisiestos.
    if years[i] % 4 == 0:
        ds_year[i]["dayofyear"] = range(1, 366)

# Cargamos la información de desfase para cada año.
shift = pd.read_csv(path_r + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True)
# Fecha pivote del desfase. 
delta = shift.max()
shift = shift.max() - shift

# Alineamos las series de tiempo.
for i in range( len(ds_year) ):
    ds_year[i] = ds_year[i].shift( { "dayofyear": shift.iloc[i] } )

# Concatenamos y promediamos por dayofyear.
composite = xr.concat(ds_year, dim = "concat")
composite = composite.mean("concat")
# Centramos el índice a la fecha de onset.
composite = composite.sel( dayofyear = slice( delta - 60, delta + 60 ) )
composite["dayofyear"] = range(-60, 61)
# Establecemos un CRS.
composite = composite.rio.write_crs(4326)

# Guardamos el netcdf.
composite.to_netcdf( path_r + "onset_composite_olr" + "_mean_" 
    + str(d)+ "_dias.nc" )


composite

<xarray.Dataset>
Dimensions:      (latitude: 161, longitude: 281, dayofyear: 121)
Coordinates:
    step         timedelta64[ns] 00:00:00
    surface      int64 0
  * latitude     (latitude) float64 40.0 39.75 39.5 39.25 ... 0.75 0.5 0.25 0.0
  * longitude    (longitude) float64 -120.0 -119.8 -119.5 ... -50.5 -50.25 -50.0
  * dayofyear    (dayofyear) int64 -60 -59 -58 -57 -56 -55 ... 55 56 57 58 59 60
    spatial_ref  int64 0
Data variables:
    mtnlwrf      (dayofyear, latitude, longitude) float64 -226.4 ... -269.6